# Atelier 2 - Détection des Opérations Juridiques
par Sebastien Meyer et Marie Tcheng 

Dans cet atelier, vous allez détecter automatiquement des **opérations juridiques** dans un arrêté préfectoral à l'aide d'un **LLM (Large Language Model)**.

## Contexte

Un arrêté préfectoral peut **modifier**, **abroger** ou **compléter** une partie d'un arrêté antérieur, ainsi que l'arrêté entier. Par exemple :
- **Abrogation** : "L'arrêté du 2023-05-10 est abrogé".
- **Modification** : "L'article 2.1 de l'arrêté du 2023-05-10 est remplacé par les dispositions suivantes..."
- **Ajout** : "Il est ajouté à l'arrêté du 2023-05-10 un article 5 ainsi rédigé..."

L'objectif est d'**extraire automatiquement** ces opérations sous forme structurée pour pouvoir ensuite reconstituer l'état consolidé d'un permis.

## Plan du TP

0. **Exercice découverte** : détecter des opérations avec un prompt libre
1. **Définir la classe `Operation`** : structure de données pour représenter une opération
2. **Prompt Engineering** : concevoir un prompt structuré pour détecter les opérations

---

## 0. Exercice découverte : détecter des opérations juridiques

Avant de définir des structures de données, commençons par **comprendre le problème**.

Vous allez recevoir des extraits d'arrêtés préfectoraux qui modifient des arrêtés antérieurs. Votre mission : écrire un prompt pour qu'un LLM détecte automatiquement ces modifications.

### 0.1 Exemples d'arrêtés

Voici 4 extraits d'arrêtés modificateurs :

In [5]:
# Exemple 1 : ABROGATION complète
exemple_1 = """
Extrait de l'arrêté du 8 janvier 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>L'arrêté préfectoral du 5 oct. 2023 est abrogé.</p>
</section>
"""

# Exemple 2 : MODIFICATION d'un article
exemple_2 = """
Extrait de l'arrêté du 12 septembre 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>Les dispositions de l'article 2.1 de l'arrêté du 5 octobre 2023 sont abrogées et remplacées par le contenu suivant :</p>
    <p>Le débit maximal de prélèvement d'eau est fixé à 150 m³/h.</p>
</section>
"""

# Exemple 3 : AJOUT d'un nouvel article
exemple_3 = """
Extrait de l'arrêté du 23/01/2023 :
<section data-spec="section" data-number="2">
    <h2>Article 2</h2>
    <p>Il est ajouté à l'arrêté du 2022-12-15 un article 4.3 ainsi rédigé :</p>
    <p>L'exploitant doit installer un dispositif de mesure du débit avant le 31 décembre 2026.</p>
</section>
"""

# Exemple 4 : MODIFICATION d'une partie spécifique
exemple_4 = """
Extrait de l'arrêté du 12 septembre 2024 :
<section data-spec="section" data-number="3.1">
    <h2>Article 3.1</h2>
    <p>Le tableau de l'article 3 de l'arrêté du 5 octobre 2023 est remplacé par le tableau suivant :</p>
    <table>
        <tr><th>Paramètre</th><th>Valeur limite</th></tr>
        <tr><td>pH</td><td>6.5 - 8.5</td></tr>
        <tr><td>DCO</td><td>125 mg/L</td></tr>
    </table>
</section>
"""


# Exemple 5 : Plusieurs opérations dans un extrait 
exemple_5 = """
Extrait de l'arrêté du 4 mars 2025 :
<section data-spec="section" data-number="4">
    <h2>Article 4</h2>
    <p>Les dispositions suivantes de l'arrêté du 12 septembre 2024 sont abrogées :</p>
    ::before 
    <ul>
        <li> - la première ligne de l' article 5.3.3; </li>
        <li> - l'article 6.7 intitulé «Déchets produits par l'établissement »; </li>
        <li> - la colonne n°1 du tableau de l'article 3.4. </li>
    </ul>
</section>
"""

print("5 exemples chargés")

5 exemples chargés


### 0.2 Exercice : Écrire un prompt libre

**Votre mission :** Écrivez un prompt qui demande au LLM de détecter les opérations juridiques dans ces extraits.

**Pour l'instant, ne vous préoccupez pas du format de sortie** - écrivez simplement ce qui vous semble naturel.

In [6]:
def mon_premier_prompt(html_content: str) -> str:
    """
    Votre prompt libre pour détecter les opérations.
    """
    # TODO : Écrivez votre prompt ici
    prompt = f"""
Voici un extrait d'arrêté :
{html_content}

A REMPLIR
"""
    return prompt

# Testez sur l'exemple 1
print(mon_premier_prompt(exemple_1))


Voici un extrait d'arrêté :

Extrait de l'arrêté du 8 janvier 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>L'arrêté préfectoral du 5 oct. 2023 est abrogé.</p>
</section>


A REMPLIR



### 0.3 Tester avec le LLM

Maintenant, testons votre prompt avec un vrai LLM (Mistral AI).

**Configuration :** Assurez-vous d'avoir une clé API dans un fichier `.env` :
```
MISTRAL_API_KEY=votre_clé_ici
```

In [7]:
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()
if os.getenv("MISTRAL_API_KEY"):
    client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))
    print("✓ LLM configuré")


def call_llm(prompt: str, model: str = "mistral-large-latest") -> str:
    """Appelle l'API Mistral."""
    # TODO : remplir 
    pass

✓ LLM configuré


Utilisez la cellule suivante pour tester directement votre code.

In [8]:
# Testez votre prompt sur les 4 exemples
print("=" * 60)
print("EXEMPLE 1 - Abrogation")
print("=" * 60)
response_1 = call_llm(mon_premier_prompt(exemple_1))
print(response_1)

print("\n" + "=" * 60)
print("EXEMPLE 2 - Modification")
print("=" * 60)
response_2 = call_llm(mon_premier_prompt(exemple_2))
print(response_2)

print("\n" + "=" * 60)
print("EXEMPLE 3 - Ajout")
print("=" * 60)
response_3 = call_llm(mon_premier_prompt(exemple_3))
print(response_3)

print("\n" + "=" * 60)
print("EXEMPLE 4 - Modification tableau")
print("=" * 60)
response_4 = call_llm(mon_premier_prompt(exemple_4))
print(response_4)

print("\n" + "=" * 60)
print("EXEMPLE 4 - Modification tableau")
print("=" * 60)
response_4 = call_llm(mon_premier_prompt(exemple_4))
print(response_4)

print("\n" + "=" * 60)
print("EXEMPLE 5 - Plusieurs opérations")
print("=" * 60)
response_5 = call_llm(mon_premier_prompt(exemple_5))
print(response_5)

EXEMPLE 1 - Abrogation
None

EXEMPLE 2 - Modification
None

EXEMPLE 3 - Ajout
None

EXEMPLE 4 - Modification tableau
None

EXEMPLE 4 - Modification tableau
None

EXEMPLE 4 - Modification tableau
None


### 0.4 Constat

**Observez les résultats :** Les réponses du LLM sont probablement :
- ✓ Correctes sur le fond
- ✗ **Différentes en format** (phrases, listes, structures variables)
- ✗ **Difficiles à parser automatiquement** dans un programme

**Problème :** Pour traiter automatiquement ces opérations (par exemple, appliquer les modifications pour reconstituer l'arrêté consolidé), on a besoin d'un **format structuré et prévisible**.

**Solution :** Définir des objets Python (classes) qui représentent ces opérations, puis imposer au LLM de respecter ce format (JSON).

---

## 1. Définition de la classe `Operation`

Une opération juridique contient plusieurs informations :
- **Type** : ADD, REMOVE, REPLACE
- **Source** : l'article de l'arrêté modificateur qui contient l'opération
- **Target** : l'arrêté et l'article à modifier
- **Operand** : le nouveau contenu (pour ADD et REPLACE)
- **SubTarget** : la partie précise à modifier (ex: "première phrase", "le tableau")

### Exercice 1.1 : Définir les types d'opérations

Créez une énumération `OperationType` avec les valeurs : `ADD`, `REMOVE`, `REPLACE`

In [9]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel, field_validator
import re

# TODO : Créer l'enum OperationType
class OperationType(Enum):
    pass  # À compléter

Utilisez la cellule suivante pour tester directement votre code.

In [10]:
# Test rapide
print(OperationType.ADD)

AttributeError: type object 'OperationType' has no attribute 'ADD'

### Exercice 1.2 : Définir la classe `NodeId`

Un `NodeId` identifie de manière unique un article dans un arrêté. Il contient :
- `arrete_id` : la date de l'arrêté au format "YYYY-MM-DD" (ex: "2023-05-10")
- `article_id` : le numéro de l'article (ex: "2.1.3", "ALL", "NEW:3.4")

NB : Les cas "ALL" et "NEW:" correspondent respectivement au cas où une opération concerne tous les articles d'un arrêté (pour une abrogation de l'arrêté entier par exemple), et le cas où l'article cible n'existe pas encore (lors de la création d'un article). 

Complétez la classe ci-dessous :

In [11]:
class NodeId(BaseModel):
    """Identifiant unique d'un nœud composé de l'ID de l'arrêté et de l'ID de l'article"""
    # TODO : Ajouter les champs arrete_id et article_id
    # TODO : ajouter un @field_validator pour valider les formats. arrete_id : YYYY-MM-DD, article_id : numérique ou "ALL" ou "NEW:x.x"
    pass

Utilisez les cellules suivantes pour tester directement votre code.

In [ ]:
# Tests à valider 
node = NodeId(arrete_id="2023-05-10", article_id="2.1")
print(f"Test validé : NodeId: {node.arrete_id}#{node.article_id}")
node = NodeId(arrete_id="2023-12-31", article_id="ALL")
print(f"Test validé : NodeId: {node.arrete_id}#{node.article_id}")
node = NodeId(arrete_id="2022-01-15", article_id="NEW:1.1")
print(f"Test validé : NodeId: {node.arrete_id}#{node.article_id}")

In [ ]:
node_invalid = NodeId(arrete_id="2023-15-10", article_id="deux")  # Doit lever une erreur de format

In [ ]:
node_invalid = NodeId(arrete_id="15 octobre 2023", article_id="2.1")  # Doit lever une erreur de format

### Exercice 1.3 : Définir la classe `Operation`

A partir de la définition de l'opération ci-dessus et en utilisant les types OperationType et NodeId créés ci-dessus,

complétez la classe ci-dessous en ajoutant tous les champs nécessaires :

In [ ]:
class Operation(BaseModel):
    """Représente une opération juridique détectée"""
    # TODO : Ajouter les champs nécessaires (id, source_id, target_id, operation_type, operand, sub_target)
    pass

Utilisez les cellules suivantes pour tester directement votre code.

In [ ]:
operation = Operation(
     id="op_001",
     source_id=NodeId(arrete_id="2024-01-8", article_id="1"),
     target_id=NodeId(arrete_id="2023-05-10", article_id="ALL"),
     operation_type=OperationType.REMOVE
)
print(operation.model_dump_json(indent=2))

In [ ]:
operation = Operation(
     id="op_004",
     source_id=NodeId(arrete_id="2024-09-12", article_id="3.1"),
     target_id=NodeId(arrete_id="2023-05-10", article_id="2.1"),
     operation_type=OperationType.REPLACE,
     operand="<table> <tr><th>Paramètre</th><th>Valeur limite</th></tr><tr><td>pH</td><td>6.5 - 8.5</td></tr><tr><td>DCO</td><td>125 mg/L</td></tr> </table>",
     sub_target="le tableau"
)
print(operation.model_dump_json(indent=2))

## 2. Détection des opérations

Maintenant que nous avons défini la structure de données pour représenter les opérations, nous allons créer un prompt pour qu'un LLM détecte automatiquement ces opérations dans un extrait d'arrêté de manière à pouvoir parser et convertir au format Operation par la suite.

Le prompt doit :
- Expliquer les 3 types d'opérations (ADD, REMOVE, REPLACE).
- Préciser les règles de parsing des dates et numéros d'article.
- Demander un format JSON structuré avec les bons champs de sortie.

### Exercice 2.1 : Identifier manuellement l'opération
Identifions manuellement une opération définie dans un extrait HTML. Reprenons l'exemple 2 : 

In [13]:
print(exemple_1)


Extrait de l'arrêté du 8 janvier 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>L'arrêté préfectoral du 5 oct. 2023 est abrogé.</p>
</section>



En lisant le HTML ci-dessus, répondez aux questions :

1. **Type d'opération** : ADD, REMOVE ou REPLACE ?
2. **Source** : Quel est l'article source (celui qui contient l'opération) ?
3. **Target arrêté** : Quelle est la date de l'arrêté cible ?
4. **Target article** : Quel article est modifié ?
5. **Operand** : Quel est le nouveau contenu ?

Écrivez vos réponses dans la cellule ci-dessous :

In [14]:
# Vos réponses ici
reponses = {
    "operation_type": "???",  # À compléter
    "source_article": "???",
    "target_arrete": "???",
    "target_article": "???",
    "operand": "???"
}

print(reponses)

{'operation_type': '???', 'source_article': '???', 'target_arrete': '???', 'target_article': '???', 'operand': '???'}


### Exercice 2.2 : Concevoir le prompt

Vous allez maintenant créer un prompt pour qu'un LLM détecte automatiquement ces informations.

**Consignes importantes pour le prompt :**
- Demander une réponse au **format JSON**
- Préciser les **3 types d'opérations** (ADD, REMOVE, REPLACE)
- Expliquer le **format de chaque champ**
- Donner des **exemples** pour chaque type
- Spécifier de retourner `[]` si aucune opération n'est détectée

Complétez la fonction ci-dessous :


In [15]:
def create_detection_prompt(html_content: str) -> str:
    """
    Crée un prompt pour détecter les opérations juridiques dans un extrait HTML.
    
    Args:
        html_content: Le contenu HTML de l'arrêté
        
    Returns:
        Le prompt complet à envoyer au LLM
    """
    prompt = f"""
Voici un extrait HTML d'arrêté préfectoral :
\"\"\"{html_content}\"\"\"

Ta tâche : identifier toutes les opérations juridiques (modifications, ajouts, abrogations).

# TODO : Compléter le prompt avec :
# - Instructions claires
# - Format JSON attendu
# - Exemples pour chaque type d'opération
# - Consignes sur les cas limites

Réponds avec une liste JSON uniquement.
"""
    return prompt

# Test du prompt
prompt = create_detection_prompt(exemple_1)
print(prompt[:500] + "...")



Voici un extrait HTML d'arrêté préfectoral :
"""
Extrait de l'arrêté du 8 janvier 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>L'arrêté préfectoral du 5 oct. 2023 est abrogé.</p>
</section>
"""

Ta tâche : identifier toutes les opérations juridiques (modifications, ajouts, abrogations).

# TODO : Compléter le prompt avec :
# - Instructions claires
# - Format JSON attendu
# - Exemples pour chaque type d'opération
# - Consignes sur les cas limites

Réponds av...


### 2.3 Tester votre nouveau prompt sur Mistral 
Nous allons maintenant tester le prompt avec l'API Mistral AI.
Rappel : vous devez avoir créé un fichier `.env` avec :
```
MISTRAL_API_KEY=votre_clé_ici
```
Executez cette cellule pour tester sur l'exemple 1 : 

In [17]:
# Test
import json
try:
    prompt = create_detection_prompt(exemple_1)
    response = call_llm(prompt)
    print("Réponse du LLM :")
    print(response)
    
    # Parser le JSON
    operations = json.loads(response)
    print(f"\n✓ {len(operations)} opération(s) détectée(s)")
    
except Exception as e:
    print(f"Erreur : {e}")

Réponse du LLM :
None
Erreur : name 'json' is not defined


### Exercice 2.3 : Améliorer le prompt

Le prompt peut être amélioré ! Testez différentes formulations et comparez les résultats.

**Pistes d'amélioration :**
- Ajouter des **exemples concrets** dans le prompt
- Préciser le format des **marqueurs de contenu** (start_marker, end_marker)
- Gérer les **cas spéciaux** (annexes, articles "ALL")
- Ajouter des **validations** (format de date, numéro d'article)

Modifiez le prompt et testez à nouveau :


In [ ]:
# Sorties attendues pour validation
expected_outputs = {
    "exemple_1": [
        {
            "operation_type": "REMOVE",
            "source_arrete": "2024-01-08",
            "source_article": "1",
            "target_arrete": "2023-05-10",
            "target_article": "ALL"
        }
    ],
    "exemple_2": [
        {
            "operation_type": "REPLACE",
            "source_arrete": "2024-09-12",
            "source_article": "1",
            "target_arrete": "2023-10-05",
            "target_article": "2.1",
            "operand" : "<p>Le débit maximal de prélèvement d'eau est fixé à 150 m³/h.</p>", 
        }
    ],
    "exemple_3": [
        {
            "operation_type": "ADD",
            "source_arrete": "2023-01-23",
            "source_article": "2",
            "target_arrete": "2022-12-15",
            "target_article": "NEW:4.3",
            "operand": "<p>L'exploitant doit installer un dispositif de mesure du débit avant le 31 décembre 2026.</p>"
        }
    ],
    "exemple_4": [
        {
            "operation_type": "REPLACE",
            "source_arrete": "2024-09-12",
            "source_article": "3.1",
            "target_arrete": "2023-10-05",
            "target_article": "3",
            "sub_target": "le tableau",
            "operand": "<table> <tr><th>Paramètre</th><th>Valeur limite</th></tr> <tr><td>pH</td><td>6.5 - 8.5</td></tr> <tr><td>DCO</td><td>125 mg/L</td></tr>  </table>",
        }
    ], 
    "exemple_5":[
        {
            "operation_type": "REMOVE",
            "source_arrete": "2024-09-12",
            "source_article": "4",
            "target_arrete": "2024-09-12",
            "target_article": "5.3.3",
            "sub_target": "la première ligne",
        }, 
        {
            "operation_type": "REMOVE",
            "source_arrete": "2024-09-12",
            "source_article": "4",
            "target_arrete": "2024-09-12",
            "target_article": "6.7"
        }, 
        {
            "operation_type": "REMOVE",
            "source_arrete": "2024-09-12",
            "source_article": "4",
            "target_arrete": "2024-09-12",
            "target_article": "3.4",
            "sub_target": "colonne n°1 du tableau",
        }
    ]
}

print("✓ Sorties attendues chargées")

In [ ]:
# Testez votre prompt sur les 4 exemples
exemples = {
    "exemple_1": exemple_1,
    "exemple_2": exemple_2,
    "exemple_3": exemple_3,
    "exemple_4": exemple_4
}

resultats = {}

for nom, html in exemples.items():
    print(f"\n{'='*60}")
    print(f"Test : {nom}")
    print(f"{'='*60}")
    
    try:
        prompt = create_detection_prompt(html)
        response = call_llm(prompt)
        operations = json.loads(response)
        resultats[nom] = operations
        
        print(f"✓ Réponse parsée : {len(operations)} opération(s)")
        print(json.dumps(operations, indent=2, ensure_ascii=False))
        
    except json.JSONDecodeError as e:
        print(f"✗ Erreur de parsing JSON : {e}")
        print(f"Réponse brute : {response[:200]}...")
    except Exception as e:
        print(f"✗ Erreur : {e}")

In [ ]:
# Validation automatique des résultats
def valider_detection(resultats, expected):
    """Compare les résultats avec les sorties attendues."""
    score = 0
    total = len(expected)
    
    for nom, expected_ops in expected.items():
        if nom not in resultats:
            print(f"✗ {nom} : non testé")
            continue
            
        result_ops = resultats[nom]
        
        if len(result_ops) != len(expected_ops):
            print(f"✗ {nom} : {len(result_ops)} opération(s) au lieu de {len(expected_ops)}")
            continue
        
        # Vérifier les champs essentiels
        op_result = result_ops[0]
        op_expected = expected_ops[0]
        
        checks = {
            "operation_type": op_result.get("operation_type") == op_expected["operation_type"],
            "target_arrete": op_result.get("target_arrete") == op_expected["target_arrete"],
            "target_article": op_result.get("target_article") == op_expected["target_article"],
        }
        
        if all(checks.values()):
            print(f"✓ {nom} : correct")
            score += 1
        else:
            errors = [k for k, v in checks.items() if not v]
            print(f"✗ {nom} : erreurs dans {errors}")
    
    print(f"\n{'='*60}")
    print(f"Score : {score}/{total}")
    if score == total:
        print("🎉 Parfait ! Votre prompt détecte correctement toutes les opérations.")
    elif score >= total * 0.75:
        print("👍 Bon travail ! Quelques ajustements à faire.")
    else:
        print("💡 Continuez à améliorer votre prompt.")
    
    return score == total

valider_detection(resultats, expected_outputs)

### Réflexion

**Comparez avec l'exercice 0 :**
- ✓ Les réponses sont maintenant **structurées** (JSON)
- ✓ Faciles à **parser** automatiquement pour les convertir au format Operation
- ✓ **Consistantes** d'un exemple à l'autre

**Remarque importante :** Pour résoudre les opérations et appliquer les modifications, il faudra parser le `subtarget` (ex: `"rubrique": "1432-2.a.1"`). Cette tâche n'est pas toujours faisable à la main (format complexe, variabilité), donc on réutilisera le LLM pour cette étape.

Vous pouvez désormais tester sur un réel arrêté prefectoral. Récupérez le HTML à la sortie et adaptez 

Quelques exemples :
- 2 modifications à détecter : https://mte-dgpr.github.io/arretify/examples/arretes_html/arretes_icpe/0005804239/2014-01-09_ap%20prescriptions%20compl%C3%A9mentaires_20140109_APCModifInstallationHuilesUsagees.html
- 15 modifications à détecter : https://mte-dgpr.github.io/arretify/examples/arretes_html/arretes_icpe/0005804239/2024-09-27_ap%20prescriptions%20compl%C3%A9mentaires_Arr%C3%AAt%C3%A9%20compl%C3%A9mentaire%20relatif%20au%20projet%20de%20r%C3%A9utilisation%20des%20eaux%20us%C3%A9es.html
- 5 modifications à détecter : https://mte-dgpr.github.io/arretify/examples/arretes_html/arretes_icpe/0005804239/2023-12-04_ap%20prescriptions%20compl%C3%A9mentaires_AP%20du%2004.12.2023_OSILUB%20%C3%A0%20Gonfreville-l'Orcher.html

### Exercice 3 : Convertir le résultat du LLM en Operation

Maintenant que vous avez obtenu un résultat JSON structuré, créez une fonction qui convertit ce résultat en objet `Operation`.

In [ ]:
def llm_result_to_operation(llm_result: dict) -> Operation:
    """
    Convertit le résultat JSON du LLM en objet Operation.
    
    Args:
        llm_result: Dictionnaire contenant les champs suivants:
            - operation_type: str (add, delete, replace)
            - target_type: str (rubrique, article, prescription, etc.)
            - subtarget: str (identifiant de la cible)
            - operand: str (contenu à ajouter/remplacer) ou None pour delete
    
    Returns:
        Operation: L'objet Operation correspondant
    """
    # À COMPLÉTER
    pass

# Test
exemple_llm = {
    "operation_type": "replace",
    "target_type": "rubrique",
    "subtarget": "1432-2.a.1",
    "operand": "La quantité maximale de produits inflammables est fixée à 50 tonnes."
}

op = llm_result_to_operation(exemple_llm)
print(f"Type: {op.type}")
print(f"Target: {op.target}")
print(f"Operand: {op.operand}")

---

## Conclusion

Dans ce notebook, vous avez exploré la détection automatique d'opérations dans les arrêtés préfectoraux :

1. **Détection naïve** : Premier contact avec la complexité du problème
2. **Prompting structuré** : Amélioration via des instructions précises et des exemples
3. **Few-shot learning** : Boost de performance avec des exemples concrets
4. **Conversion en Operation** : Transformation du JSON en objets manipulables

**Prochaines étapes** : Utiliser ces opérations détectées pour mettre à jour automatiquement les prescriptions dans la base de données.